<a href="https://colab.research.google.com/github/Mvgs0005/cod/blob/main/examples/license-plate-ocr-notebook/license_plate_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# License Plate Detection and OCR Processing




## Installation and Setup



In [ ]:
!pip install landingai gdown

!gdown "https://drive.google.com/uc?id=16iwE7mcz9zHqKCw2ilx0QEwSCjDdXEW4" -O /tmp/license-plates.mov

<a name="frame-extraction"></a>
## Frame Extraction




In [ ]:
from IPython.display import Video
Video("https://drive.google.com/uc?export=view&id=16iwE7mcz9zHqKCw2ilx0QEwSCjDdXEW4", width=512)

In [ ]:
from landingai.pipeline.image_source import VideoFile

# Replace 'path_to_video_file' with the actual path to your video file
# video_file_path = '/Users/whit_blodgett/Desktop/Code/landing-apps-poc/license_plate_ocr_app/IMG_2464.MOV'
video_file_path = "/tmp/license-plates.mov"

video_source = VideoFile(video_file_path, samples_per_second=1)
frames = [f.image for f in video_source]
print(f"Extracted {len(frames)} frames from the above video")

<a name="license-plate-detection"></a>

## License Plate Detection


In [ ]:
from landingai.predict import Predictor
from landingai import visualize


def detect_license_plates(frames):
    bounding_boxes = []
    overlayed_frames = []
    api_key = "land_sk_aMemWbpd41yXnQ0tXvZMh59ISgRuKNRKjJEIUHnkiH32NBJAwf"
    model_endpoint = "e001c156-5de0-43f3-9991-f19699b31202"
    predictor = Predictor(model_endpoint, api_key=api_key)

    for frame in frames:
        prediction = predictor.predict(frame)
        # store predictions in a list
        overlay = visualize.overlay_predictions(prediction, frame)
        bounding_boxes.append(prediction)
        overlayed_frames.append(overlay)

    return bounding_boxes, overlayed_frames

bounding_boxes, overlayed_frames = detect_license_plates(frames)

# show 5 overlayed frames
for i, frame in enumerate(overlayed_frames):
    if len(bounding_boxes[i]) == 0:
        continue
    display(frame)

<a name="image-cropping"></a>

## Image Cropping


In [ ]:
from landingai.postprocess import crop

# cropping the license plate
cropped_imgs = []
for frame, bboxes in zip(frames, bounding_boxes):
    cropped_imgs.append(crop(bboxes, frame))

print(len(cropped_imgs))
# show 5 overlayed frames
for i, cropped in enumerate(cropped_imgs):
    if len(cropped) == 0:
        continue
    for plate in cropped:
        display(plate)

<a name="ocr-and-data-retrieval"></a>



In [ ]:
from landingai.predict import OcrPredictor

# NOTE: The API key below has a rate limit. Use an API key from your own LandingLens account for production use.
API_KEY = "land_sk_WVYwP00xA3iXely2vuar6YUDZ3MJT9yLX6oW5noUkwICzYLiDV"
ocr_predictor = OcrPredictor(api_key=API_KEY)

ocr_preds = []
overlayed_ocr = []
print(cropped_imgs[0])
for frame in cropped_imgs:
    for plate in frame:
        ocr_pred = ocr_predictor.predict(plate)
        ocr_preds.append(ocr_pred)
        overlay = visualize.overlay_predictions(ocr_pred, plate)
        overlayed_ocr.append(overlay)
print(ocr_preds)
for frame, ocr_pred in zip(overlayed_ocr, ocr_preds):
    if len(ocr_pred) == 0:
        continue
    display(frame)
    for text in ocr_pred:
        print(text.text)